In [1]:
import pandas as pd

In [2]:
from NetworkIntrusionDetection import logger
from NetworkIntrusionDetection.entity.config_entity import FEConfig
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/home/sanskar/Desktop/network_intrusion_det/artifacts/data_ingestion/US-AirForce-LAN-Network-Intrusion/data/data.csv')


In [4]:
le = LabelEncoder()
df['class'] = le.fit_transform(df['class'])
df['class'].unique()

array([1, 0])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('class', axis=1), df['class'], test_size=0.2, random_state=42)

In [6]:
num_cols = X_train.select_dtypes(exclude="object").columns
cat_cols = X_train.select_dtypes(include="object").columns
        
# convert X data to dataframe format
X_train = pd.DataFrame(X_train, columns=X_train.columns)
X_test = pd.DataFrame(X_test, columns=X_test.columns)

In [7]:
X_train["service"].unique()

array(['eco_i', 'private', 'other', 'bgp', 'http', 'ecr_i', 'domain_u',
       'smtp', 'ldap', 'auth', 'daytime', 'ftp', 'ftp_data', 'exec',
       'time', 'finger', 'uucp_path', 'netstat', 'link', 'urp_i',
       'telnet', 'whois', 'csnet_ns', 'klogin', 'imap4', 'ctf', 'efs',
       'sunrpc', 'Z39_50', 'domain', 'echo', 'printer', 'netbios_ns',
       'discard', 'nnsp', 'ntp_u', 'login', 'hostnames', 'courier',
       'iso_tsap', 'netbios_dgm', 'uucp', 'http_443', 'ssh', 'vmnet',
       'supdup', 'gopher', 'sql_net', 'name', 'systat', 'kshell', 'X11',
       'mtp', 'pop_2', 'shell', 'pop_3', 'netbios_ssn', 'IRC', 'red_i',
       'nntp', 'remote_job', 'rje', 'pm_dump', 'tim_i', 'urh_i'],
      dtype=object)

In [8]:
X_test["service"].unique()

array(['ecr_i', 'http', 'private', 'domain_u', 'smtp', 'Z39_50', 'other',
       'echo', 'ftp_data', 'name', 'eco_i', 'ctf', 'bgp', 'exec',
       'courier', 'IRC', 'telnet', 'supdup', 'klogin', 'daytime', 'link',
       'discard', 'systat', 'login', 'finger', 'pop_3', 'gopher', 'ldap',
       'nntp', 'efs', 'netbios_ssn', 'hostnames', 'vmnet', 'ntp_u',
       'time', 'netbios_dgm', 'auth', 'ftp', 'http_443', 'kshell', 'nnsp',
       'imap4', 'remote_job', 'uucp_path', 'csnet_ns', 'sql_net',
       'domain', 'netbios_ns', 'uucp', 'urh_i', 'iso_tsap', 'sunrpc',
       'rje', 'ssh', 'mtp', 'whois', 'netstat', 'http_8001', 'urp_i',
       'shell', 'X11', 'red_i', 'pop_2'], dtype=object)

In [9]:
# le = LabelEncoder()
# for col in cat_cols:
#     X_train[col] = le.fit_transform(X_train[col])
#     X_test[col] = le.transform(X_test[col])
            

# Define all possible categories based on the unique values in both training and test sets for all categorical columns
all_categories = set(X_train["service"].unique()).union(set(X_test["service"].unique()))
all_categories = all_categories.union(set(X_train["protocol_type"].unique())).union(set(X_test["protocol_type"].unique()))
all_categories = all_categories.union(set(X_train["flag"].unique())).union(set(X_test["flag"].unique()))

# Initialize LabelEncoder
le = LabelEncoder()

# Fit LabelEncoder on all possible categories
le.fit(list(all_categories))

# Transform categorical columns in both training and test sets
for col in cat_cols:
    X_train[col] = le.transform(X_train[col])
    X_test[col] = le.transform(X_test[col])


In [10]:
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [11]:
temp = X_train

In [12]:
X_test

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
19064,-0.115524,36,25,10,-0.009858,-0.040372,-0.009962,-0.09257,-0.007044,-0.093748,...,0.730195,1.267439,1.071796,-0.442064,2.762381,-0.290057,-0.638974,-0.624452,-0.386320,-0.375081
11127,-0.115524,69,33,2,-0.010241,-0.040372,-0.009962,-0.09257,-0.007044,-0.093748,...,-1.411867,0.127411,1.071796,-0.442064,-0.414835,1.075570,-0.638974,-0.601978,2.649641,2.362208
6517,-0.115524,69,58,6,-0.010241,-0.040372,-0.009962,-0.09257,-0.007044,-0.093748,...,0.730195,-1.012617,-1.134746,-0.120694,-0.479676,-0.290057,1.611411,1.622972,-0.386320,-0.375081
2973,-0.115524,73,22,10,-0.010225,-0.038874,-0.009962,-0.09257,-0.007044,-0.093748,...,-1.573532,-0.931186,-0.176349,0.147115,-0.349993,-0.290057,-0.638974,-0.624452,-0.386320,-0.375081
13339,-0.115524,69,33,6,-0.010241,-0.040372,-0.009962,-0.09257,-0.007044,-0.093748,...,-1.725093,-0.931186,1.071796,-0.442064,-0.220311,-0.290057,1.611411,1.622972,-0.386320,-0.375081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16632,-0.115524,69,33,10,-0.010161,0.270003,-0.009962,-0.09257,-0.007044,-0.093748,...,-1.765510,1.267439,1.071796,-0.442064,-0.090629,0.074110,-0.638974,-0.624452,-0.386320,-0.375081
360,-0.115524,69,63,10,-0.009838,-0.036404,-0.009962,-0.09257,-0.007044,-0.093748,...,0.639259,-0.460698,-0.622115,1.111225,-0.479676,-0.016932,-0.638974,-0.624452,0.462443,-0.375081
14748,-0.115524,69,33,10,-0.010095,0.033773,-0.009962,-0.09257,-0.007044,-0.093748,...,-0.856143,1.267439,1.071796,-0.442064,-0.447255,0.165152,-0.638974,-0.624452,-0.386320,-0.375081
2910,-0.115524,69,33,10,-0.010187,0.112477,-0.009962,-0.09257,-0.007044,-0.093748,...,-1.179474,1.267439,1.071796,-0.442064,-0.414835,-0.107974,-0.638974,-0.624452,-0.386320,-0.343618


In [13]:
# apply select k best
selector = SelectKBest(f_classif, k=10)
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)

/home/sanskar/anaconda3/envs/NIDvenv/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [19 20] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/sanskar/anaconda3/envs/NIDvenv/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [14]:
feature_importances = selector.scores_
# Get indices of top 10 features by descending importance
top_10_indices = np.argsort(feature_importances)[::-1][:10]  # Descending order, top 10

# Select features based on the top 10 indices
selected_features = temp.columns[top_10_indices]

In [15]:
selected_features

Index(['is_host_login', 'num_outbound_cmds', 'same_srv_rate',
       'dst_host_srv_count', 'dst_host_same_srv_rate', 'logged_in',
       'dst_host_srv_serror_rate', 'dst_host_serror_rate', 'serror_rate',
       'flag', 'srv_serror_rate', 'count', 'dst_host_count', 'service',
       'dst_host_rerror_rate'],
      dtype='object')